# 21.1 Garbage Collection

What is garbage collection? Explain it in the context of Python

### Python uses two strategies for memory allocation:

- #### Reference counting

    -  Reference counting works by counting the number of times an object is referenced by other objects in the system. When references to an object are removed, the reference count for an object is decremented. When the reference count becomes zero, the object is deallocated.
    
    
- #### Garbage collection

    -  As objects are created. they are assigned to generations, and younger generatrions are examined first
    - When the number of allocations vs. the number of deallocations is greater than a threshold the automatic garbage collector will run. 


### Manual Garbage Collection

In [4]:
import gc 
i = 0 
  
# create a cycle and on each iteration x as a dictionary 
# assigned to 1 
def create_cycle(): 
    x = { } 
    x[i+1] = x 
    print(x) 

#lists are cleared whenever a full collection or  
# collection of the highest generation (2) is run 
collected = gc.collect() # or gc.collect(2) 
print("Garbage collector: collected %d objects." % (collected) )
  
print("Creating cycles...")
for i in range(10): 
    create_cycle() 

collected = gc.collect() 
  
print("Garbage collector: collected %d objects." % (collected) )

Garbage collector: collected 34 objects.
Creating cycles...
{1: {...}}
{2: {...}}
{3: {...}}
{4: {...}}
{5: {...}}
{6: {...}}
{7: {...}}
{8: {...}}
{9: {...}}
{10: {...}}
Garbage collector: collected 10 objects.


# 21.2 Closure
What does the following program print and why?

In [1]:
increment_by_i = [lambda x: x + i for i in range(10)]

print(increment_by_i[3](4))

#not 7 from (i = 3 => 3) + 4

#correct 13 from (i = 10 => 9) + 4

13


In [6]:
## how to get 7

def create_increment_function(x):
    return lambda y: y + x

increment_by_i = [create_increment_function(i) for i in range(10)]

print(increment_by_i[3](4))

7


# 21.3 Shallow and Deep Copy
Describe the difference between a shallow copy and a deep copy. When is each appropiate, and when is neither appropiate? How is deep copy implemented?

### Note
Assigment in python does not copy, its simply a variable to a target

- ### Shallow Copy

    - Copies existing value with same reference to the origial object. So if you can the new copy, the original copy will also get changed
    
    
- ### Deep Copy

    - Copies values to a new object recursively with a new reference, so they are completetly separete objects in memory  

# 21.4 Iterators and Generators
What is the difference between an iterator and a generator?